In [6]:
%matplotlib inline
import json
import math
import numpy as np

import matplotlib.pyplot as plt

import subprocess
import json

import librosa

In [7]:
raw_data = subprocess.run(['ruby', '../../ableton_extractor/extract.rb', 'setncs.als', '2-Audio', '1-Audio'], stdout=subprocess.PIPE).stdout
data = json.loads(raw_data)

In [8]:
curves = {
    # Constant power
    '2': lambda x: math.cos(x * math.pi/2)
}

curve_fn = np.vectorize(curves[data['crossfade']['curve']])
cf_events = data['crossfade']['events']

In [5]:
from itertools import takewhile

cf_val = 0

events = list(cf_events[1:])

xfades = []

while len(events) > 0:
    xfades.append([])
    
    end_found = False
    while not end_found:
        event = events.pop(0)
        xfades[-1].append(event)
        
        end_found = event['value'] == cf_val
    
    cf_val = 1 - cf_val

In [21]:
interp = lambda x: np.interp(x, [e['time'] for e in cf_events], [e['value'] for e in cf_events])

In [22]:
x = np.linspace(0, np.max([e['time'] for e in cf_events]) + 30, 100000)

In [23]:
xf_a = lambda x: curve_fn(interp(x))
xf_b = lambda x: curve_fn(1 - interp(x))

In [25]:
def process_clip(clip):
    sr = 44100.0
    audio_data, _ = librosa.load(clip['relative_path'][:-4] + '.wav', sr=sr)
    
    start = int(clip['time'] * sr)
    offset = int(clip['offset'] * sr)
    
    start_with_offset = start - offset
    
    rng = np.arange(0, len(audio_data))

    stretcher = lambda x: np.interp(x, rng, audio_data)

    n_elems_s = int(audio_data.shape[0]/clip['stretch'])

    stretched = stretcher(np.arange(n_elems_s)*clip['stretch'])
    
    c_fns = {
        'A': xf_a,
        'B': xf_b,
    }

    final = stretched * c_fns[clip['track']]((start_with_offset/sr) + (np.arange(len(stretched))/sr))
    final[:offset] = 0
    
    if start_with_offset < 0:
        final = final[abs(start_with_offset):]
    elif start_with_offset > 0:
        final = np.pad(final, (start_with_offset, 0), 'constant')
        
    endsample = int(clip['end'] * sr)
    final[endsample:] = 0

    return final.astype(np.float32), int(sr)

In [26]:
clips = []

for clip in data['clips']:
    print(f'Processing clip {clip["name"]}')
    clips.append(process_clip(clip))

Processing clip Debris & Our Psych - Omerta _NCS Release_ ( 160kbps ).wav
Processing clip NIVIRO - The Floor Is Lava _NCS Release_ ( 160kbps ).wav
Processing clip NIVIRO - Memes [NCS Release]-hgXZQfHySDw.wav
Processing clip Kisma - Fingertips [NCS Release]-LJeiQw2RmSg.wav
Processing clip Elektronomia_-_Sky_High_[NCS_Release].wav
Processing clip Raven & Kreyn - Biscuit [NCS Release]-npG_UUUnwjI.wav
Processing clip Culture Code - Feel Again (feat. Harley Bird) [NCS Release]-YGLA_V_yZXo.wav


In [27]:
from functools import reduce

maxlen = max([len(x) for x, _ in clips])

def reducer(a, b):
    return np.pad(a, (0, maxlen - len(a)), 'constant') + np.pad(b, (0, maxlen - len(b)), 'constant')

reduced = reduce(reducer, [x for x, _ in clips])

In [28]:
librosa.output.write_wav('n_setncs.wav', reduced.astype(np.float32), 44100)